In [59]:
import tensorflow as tf
import glob, os
import numpy as np

In [60]:
folder = 'data/dogs-vs-cats/'

In [61]:
train_folder = folder + 'train/'
test_folder = folder + 'test/'

In [62]:
train_paths = glob.glob(train_folder + '*')
test_paths = glob.glob(test_folder + '*')
train_paths[-1]

'data/dogs-vs-cats/train/cat.10148.jpg'

In [63]:
filename = tf.strings.split(train_paths[0], os.sep)[-1]
label = tf.strings.split(filename, '.')[0]
label_decoded = label.numpy().decode()

In [64]:
label_decoded

'dog'

In [65]:
def get_label(file_path):
    filename = tf.strings.split(file_path, os.sep)[-1]
    label_str = tf.strings.split(filename, '.')[0]

    # Map string label to integer
    label_map = {'cat': 0, 'dog': 1}
    return label_map[label_str.numpy().decode()]


In [66]:
def get_label_tf(file_path):
    return tf.py_function(func=get_label, inp=[file_path], Tout=tf.int32)


In [67]:
def decode_img(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    return img

In [68]:
train_paths

['data/dogs-vs-cats/train/dog.8011.jpg',
 'data/dogs-vs-cats/train/cat.5077.jpg',
 'data/dogs-vs-cats/train/dog.7322.jpg',
 'data/dogs-vs-cats/train/cat.2718.jpg',
 'data/dogs-vs-cats/train/cat.10151.jpg',
 'data/dogs-vs-cats/train/cat.3406.jpg',
 'data/dogs-vs-cats/train/dog.1753.jpg',
 'data/dogs-vs-cats/train/cat.4369.jpg',
 'data/dogs-vs-cats/train/cat.7660.jpg',
 'data/dogs-vs-cats/train/dog.5535.jpg',
 'data/dogs-vs-cats/train/cat.8553.jpg',
 'data/dogs-vs-cats/train/cat.9895.jpg',
 'data/dogs-vs-cats/train/cat.1211.jpg',
 'data/dogs-vs-cats/train/dog.3144.jpg',
 'data/dogs-vs-cats/train/dog.775.jpg',
 'data/dogs-vs-cats/train/dog.11102.jpg',
 'data/dogs-vs-cats/train/cat.6218.jpg',
 'data/dogs-vs-cats/train/dog.11664.jpg',
 'data/dogs-vs-cats/train/dog.3622.jpg',
 'data/dogs-vs-cats/train/cat.1577.jpg',
 'data/dogs-vs-cats/train/cat.12020.jpg',
 'data/dogs-vs-cats/train/dog.5253.jpg',
 'data/dogs-vs-cats/train/cat.7106.jpg',
 'data/dogs-vs-cats/train/cat.8235.jpg',
 'data/dogs-v

In [69]:
def process_path(file_path):
    label = get_label_tf(file_path)
    image = decode_img(file_path)
    return image, label

## Test does not have labels.  Will need to adapt this for test file.

train_ds = tf.data.Dataset.from_tensor_slices(train_paths)
# test_ds = tf.data.Dataset.from_tensor_slices(test_paths)


In [70]:
for x in train_ds.take(5):
    print(x)

tf.Tensor(b'data/dogs-vs-cats/train/dog.8011.jpg', shape=(), dtype=string)
tf.Tensor(b'data/dogs-vs-cats/train/cat.5077.jpg', shape=(), dtype=string)
tf.Tensor(b'data/dogs-vs-cats/train/dog.7322.jpg', shape=(), dtype=string)
tf.Tensor(b'data/dogs-vs-cats/train/cat.2718.jpg', shape=(), dtype=string)
tf.Tensor(b'data/dogs-vs-cats/train/cat.10151.jpg', shape=(), dtype=string)


In [71]:
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
# test_ds = test_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
train_ds

<_ParallelMapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>

In [72]:
train_ds

<_ParallelMapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>

In [73]:
for images, labels in train_ds.take(1):
  print(images.shape)
  print(labels.numpy())

(224, 224, 3)
1


In [74]:
import sys
sys.getsizeof(train_ds)

48

In [75]:
train_ds = train_ds.batch(32).shuffle(buffer_size=100).cache().prefetch(buffer_size=tf.data.AUTOTUNE)